In [1]:
import os
import numpy as np
from nanowire.optics.simulate import Simulator
from nanowire.optics.postprocess import Simulation
from nanowire.optics.utils.utils import setup_sim
from nanowire.optics.utils.config import Config
import scipy.constants as consts
import scipy.integrate as intg
import IPython.display as disp
import matplotlib.pyplot as plt
%load_ext autoreload 
%autoreload 2

# Utility Functions

In [2]:
def integrate(arr, layer_obj, sim_proc):
    arr_slice = arr[layer_obj.slice]
    zsamps = layer_obj.iend - layer_obj.istart
    z_vals = np.linspace(0, layer_obj.thickness, zsamps)
    x_vals = np.linspace(0, sim_proc.period, sim_proc.x_samples)
    y_vals = np.linspace(0, sim_proc.period, sim_proc.y_samples)
    z_integral = intg.trapz(arr_slice, x=z_vals, axis=0)
    x_integral = intg.trapz(z_integral, x=x_vals, axis=0)
    y_integral = intg.trapz(x_integral, x=y_vals, axis=0)
    return y_integral

def compute_fluxes(sim):
    fluxes = sim.flux_dict
    total_incident_power = .5*sim.period**2/Zo*np.absolute(fluxes['Air'][0])
    total_reflected_power = .5*sim.period**2/Zo*np.absolute(fluxes['Air'][1])
    total_transmitted_power = .5*sim.period**2/Zo*np.absolute(sum(fluxes['Substrate_bottom']))
    total_absorbed_power = total_incident_power - total_reflected_power - total_transmitted_power
    print('Total Incident Power = {}'.format(total_incident_power))
    print('Total Reflected Power = {}'.format(total_reflected_power))
    print('Total Transmitted Power = {}'.format(total_transmitted_power))
    print('Total Absorbed Power = {}'.format(total_absorbed_power))
    summed_absorbed_power = 0
    abs_dict_fluxmethod = {}
    for layer, (forw_top, back_top) in fluxes.items():
        if '_bottom' in layer:
            continue
        bottom = layer+'_bottom'
        forw_bot, back_bot = fluxes[bottom] 
        print('-'*25)
        print('Layer: {}'.format(layer))
        print('Forward Top: {}'.format(forw_top))
        print('Backward Top: {}'.format(back_top))
        print('Forward Bottom: {}'.format(forw_bot))
        print('Backward Bottom: {}'.format(back_bot))
        P_in = forw_top + -1*back_bot
        P_out = forw_bot + -1*back_top
        print('Power Entering Layer: {}'.format(P_in))
        print('Power Leaving Layer: {}'.format(P_out))
        P_lost = P_in - P_out
        P_abs = .5*P_lost*(sim.period**2)/Zo
        abs_dict_fluxmethod[layer] = P_abs 
        print('Absorbed in Layer: {}'.format(P_abs))
        summed_absorbed_power += P_abs
    print('-'*25)
    print('Summed Absorption= {}'.format(summed_absorbed_power))
    return abs_dict_fluxmethod

# NW w/ Shell

In [3]:
conf = Config('AbsorptionTest.yml')
sim = Simulator(conf)
sim = setup_sim(sim)
Zo = consts.physical_constants['characteristic impedance of vacuum'][0]

In [4]:
sim.get_fluxes()

{'Air': ((8+0j), (-0.6223175327719734+0j)),
 'Air_bottom': ((8+0j), (-0.6223175327719733+0j)),
 'ITO': ((7.646975790277538+0.003485300685028281j),
  (-0.2692933230497826-0.003485300685028281j)),
 'ITO_bottom': ((7.160561661261964-0.009475598998399282j),
  (-0.2779707174415758+0.009475598998399282j)),
 'NW_AlShell': ((6.925511077701219-0.008757040538793975j),
  (-0.04292013388091296+0.008757040538793975j)),
 'NW_AlShell_bottom': ((2.479936049259686-0.014221520179471897j),
  (-0.1315622983299687+0.014221520179471897j)),
 'NW_SiO2': ((2.566272676217353-0.010733923859513661j),
  (-0.2178989252876916+0.010733923859513661j)),
 'NW_SiO2_bottom': ((2.5307544982248817-0.001534218733692104j),
  (-0.21940000157852244+0.001534218733692104j)),
 'Substrate': ((2.3113544966463353+0j), 0j),
 'Substrate_bottom': ((0.35734335906371556+0j), 0j)}

In [5]:
abs_dict_fluxmethod = compute_fluxes(sim)

Total Incident Power = 0.0006636046823595181
Total Reflected Power = 5.1621603582738056e-05
Total Transmitted Power = 2.9641840785595023e-05
Total Absorbed Power = 0.0005823412379911849
-------------------------
Layer: ITO
Forward Top: (7.646975790277538+0.003485300685028281j)
Backward Top: (-0.2692933230497826-0.003485300685028281j)
Forward Bottom: (7.160561661261964-0.009475598998399282j)
Backward Bottom: (-0.2779707174415758+0.009475598998399282j)
Power Entering Layer: (7.924946507719114-0.005990298313371001j)
Power Leaving Layer: (7.4298549843117465-0.005990298313371001j)
Absorbed in Layer: (4.106813164120451e-05+0j)
-------------------------
Layer: Substrate
Forward Top: (2.3113544966463353+0j)
Backward Top: 0j
Forward Bottom: (0.35734335906371556+0j)
Backward Bottom: 0j
Power Entering Layer: (2.3113544966463353+0j)
Power Leaving Layer: (0.35734335906371556+0j)
Absorbed in Layer: (0.00016208636753530938+0j)
-------------------------
Layer: NW_SiO2
Forward Top: (2.566272676217353-0

In [6]:
sim.get_field()

In [7]:
sim_proc = Simulation(simulator=sim)
freq = sim_proc.conf[('Simulation', 'params', 'frequency','value')]
try:
    Esq = sim_proc.data['normEsquared']
except KeyError:
    Esq = sim_proc.normEsquared()
abs_dict_intmethod = {}
for layer_name, layer_obj in sim_proc.layers.items():
    print("Layer: {}".format(layer_name))
    base_unit = sim_proc.conf[('Simulation', 'base_unit')]
    n_mat, k_mat = layer_obj.get_nk_matrix(freq)
    # n and k could be functions of space, so we need to multiply the
    # fields by n and k before integrating
    res = integrate(Esq*n_mat*k_mat,layer_obj, sim_proc)
    p_abs_imag = 2*np.pi*freq*consts.epsilon_0*res*base_unit
    abs_dict_intmethod[layer_name] = p_abs_imag
    disp.display_latex("$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV"+" = {}$".format(p_abs_imag), raw=True)

Layer: Air


Layer: ITO


Layer: NW_AlShell


Layer: NW_SiO2


Layer: Substrate


In [8]:
print(abs_dict_fluxmethod)
print(abs_dict_intmethod)

{'Air': 0j, 'NW_AlShell': (0.0003761159700046531+0j), 'ITO': (4.106813164120451e-05+0j), 'Substrate': (0.00016208636753530938+0j), 'NW_SiO2': (3.070768809982097e-06+0j)}
{'Air': 0.0, 'NW_AlShell': 0.00039358724356213935, 'ITO': 4.1088123474421656e-05, 'Substrate': 0.00015756428563003678, 'NW_SiO2': 3.4075267331699198e-06}


In [9]:
for key in abs_dict_fluxmethod.keys():
    fm = abs_dict_fluxmethod[key]
    im = abs_dict_intmethod[key]
    diff = fm - im
    try:
        pdiff = 100*abs(diff)/fm
    except ZeroDivisionError:
        pdiff = None
        pass
    print('-'*25)
    print("Layer: {}".format(key))
    print("Flux Method: {}".format(fm))
    print("Integral Method: {}".format(im))
    print("Diff: {}".format(diff))
    print("Percent Diff: {}".format(pdiff))

-------------------------
Layer: Air
Flux Method: 0j
Integral Method: 0.0
Diff: 0j
Percent Diff: None
-------------------------
Layer: NW_AlShell
Flux Method: (0.0003761159700046531+0j)
Integral Method: 0.00039358724356213935
Diff: (-1.7471273557486257e-05+0j)
Percent Diff: (4.645182590164973+0j)
-------------------------
Layer: ITO
Flux Method: (4.106813164120451e-05+0j)
Integral Method: 4.1088123474421656e-05
Diff: (-1.9991833217147207e-08+0j)
Percent Diff: (0.04867967550072082+0j)
-------------------------
Layer: Substrate
Flux Method: (0.00016208636753530938+0j)
Integral Method: 0.00015756428563003678
Diff: (4.522081905272598e-06+0j)
Percent Diff: (2.789921184634787+0j)
-------------------------
Layer: NW_SiO2
Flux Method: (3.070768809982097e-06+0j)
Integral Method: 3.4075267331699198e-06
Diff: (-3.367579231878226e-07+0j)
Percent Diff: (10.966567137621341+0j)


# NW w/ Shell w/ Lanczos

In [10]:
conf = Config('AbsorptionTest.yml')
conf[('Solver', 'LanczosSmoothing')] = True
sim = Simulator(conf)
sim = setup_sim(sim)
Zo = consts.physical_constants['characteristic impedance of vacuum'][0]

In [11]:
sim.get_fluxes()

{'Air': ((8-1.1102230246251565e-16j),
  (-0.7634567825647854+1.1102230246251565e-16j)),
 'Air_bottom': ((8+0j), (-0.7634567825647852+0j)),
 'ITO': ((7.458803815628605+0.0006962511562763618j),
  (-0.22226059819365043-0.0006962511562763618j)),
 'ITO_bottom': ((6.985538323184275-0.006973473057016066j),
  (-0.22701755175106705+0.006973473057016066j)),
 'NW_AlShell': ((6.810464916302623-0.005459676801960411j),
  (-0.05194414486951902+0.005459676801960411j)),
 'NW_AlShell_bottom': ((2.655213701216194-0.012739572446102104j),
  (-0.15086594283916177+0.012739572446102104j)),
 'NW_SiO2': ((2.739239838534732-0.013372127431107994j),
  (-0.2348920801576382+0.013372127431107994j)),
 'NW_SiO2_bottom': ((2.7071301062440085-0.005874276380139681j),
  (-0.23743488369594565+0.005874276380139681j)),
 'Substrate': ((2.469695222548072+0j), 0j),
 'Substrate_bottom': ((0.38398303284842933+0j), 0j)}

In [12]:
abs_dict_fluxmethod = compute_fluxes(sim)

Total Incident Power = 0.0006636046823595181
Total Reflected Power = 6.332918696114051e-05
Total Transmitted Power = 3.1851617318103296e-05
Total Absorbed Power = 0.0005684238780802744
-------------------------
Layer: ITO
Forward Top: (7.458803815628605+0.0006962511562763618j)
Backward Top: (-0.22226059819365043-0.0006962511562763618j)
Forward Bottom: (6.985538323184275-0.006973473057016066j)
Backward Bottom: (-0.22701755175106705+0.006973473057016066j)
Power Entering Layer: (7.685821367379672-0.0062772219007397045j)
Power Leaving Layer: (7.207798921377925-0.0062772219007397045j)
Absorbed in Layer: (3.965224167996363e-05+0j)
-------------------------
Layer: Substrate
Forward Top: (2.469695222548072+0j)
Backward Top: 0j
Forward Bottom: (0.38398303284842933+0j)
Backward Bottom: 0j
Power Entering Layer: (2.469695222548072+0j)
Power Leaving Layer: (0.38398303284842933+0j)
Absorbed in Layer: (0.0001730110468923758+0j)
-------------------------
Layer: NW_SiO2
Forward Top: (2.739239838534732-

In [13]:
sim.get_field()

In [14]:
sim_proc = Simulation(simulator=sim)
freq = sim_proc.conf[('Simulation', 'params', 'frequency','value')]
try:
    Esq = sim_proc.data['normEsquared']
except KeyError:
    Esq = sim_proc.normEsquared()
abs_dict_intmethod = {}
for layer_name, layer_obj in sim_proc.layers.items():
    print("Layer: {}".format(layer_name))
    base_unit = sim_proc.conf[('Simulation', 'base_unit')]
    n_mat, k_mat = layer_obj.get_nk_matrix(freq)
    # n and k could be functions of space, so we need to multiply the
    # fields by n and k before integrating
    res = integrate(Esq*n_mat*k_mat,layer_obj, sim_proc)
    p_abs_imag = 2*np.pi*freq*consts.epsilon_0*res*base_unit
    abs_dict_intmethod[layer_name] = p_abs_imag
    disp.display_latex("$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV"+" = {}$".format(p_abs_imag), raw=True)

Layer: Air


Layer: ITO


Layer: NW_AlShell


Layer: NW_SiO2


Layer: Substrate


In [15]:
print(abs_dict_fluxmethod)
print(abs_dict_intmethod)

{'Air': 0j, 'NW_AlShell': (0.00035288614137893857+0j), 'ITO': (3.965224167996363e-05+0j), 'Substrate': (0.0001730110468923758+0j), 'NW_SiO2': (2.874448128971978e-06+0j)}
{'Air': 0.0, 'NW_AlShell': 0.00037703838159469243, 'ITO': 3.9530936675958905e-05, 'Substrate': 0.00016864834863216508, 'NW_SiO2': 3.6733446094810126e-06}


In [16]:
for key in abs_dict_fluxmethod.keys():
    fm = abs_dict_fluxmethod[key]
    im = abs_dict_intmethod[key]
    diff = fm - im
    try:
        pdiff = 100*abs(diff)/fm
    except ZeroDivisionError:
        pdiff = None
        pass
    print('-'*25)
    print("Layer: {}".format(key))
    print("Flux Method: {}".format(fm))
    print("Integral Method: {}".format(im))
    print("Diff: {}".format(diff))
    print("Percent Diff: {}".format(pdiff))

-------------------------
Layer: Air
Flux Method: 0j
Integral Method: 0.0
Diff: 0j
Percent Diff: None
-------------------------
Layer: NW_AlShell
Flux Method: (0.00035288614137893857+0j)
Integral Method: 0.0003770383815946924
Diff: (-2.415224021575386e-05+0j)
Percent Diff: (6.8442019630968+0j)
-------------------------
Layer: ITO
Flux Method: (3.965224167996363e-05+0j)
Integral Method: 3.9530936675958905e-05
Diff: (1.213050040047264e-07+0j)
Percent Diff: (0.30592218463659293+0j)
-------------------------
Layer: Substrate
Flux Method: (0.0001730110468923758+0j)
Integral Method: 0.00016864834863216508
Diff: (4.362698260210716e-06+0j)
Percent Diff: (2.521629883509462+0j)
-------------------------
Layer: NW_SiO2
Flux Method: (2.874448128971978e-06+0j)
Integral Method: 3.6733446094810126e-06
Diff: (-7.988964805090346e-07+0j)
Percent Diff: (27.793038686516606+0j)


# NW no Shell

In [17]:
conf = Config('AbsorptionTestnoshell.yml')
sim = Simulator(conf)
sim = setup_sim(sim)
Zo = consts.physical_constants['characteristic impedance of vacuum'][0]

In [18]:
sim.get_fluxes()

{'Air': ((8+0j), (-0.9852588301655535+0j)),
 'Air_bottom': ((8+0j), (-0.9852588301655535+0j)),
 'ITO': ((7.133256784779571-0.00415199595578708j),
  (-0.1185156149453849+0.00415199595578708j)),
 'ITO_bottom': ((6.657799654778778+0.007113102097409382j),
  (-0.13981974282046136-0.007113102097409382j)),
 'NW_AlShell': ((6.744153574990113+0.010119357499265125j),
  (-0.2261736630319038-0.010119357499265125j)),
 'NW_AlShell_bottom': ((4.15627843758476+0.01853459036351636j),
  (-0.36664157753097343-0.01853459036351636j)),
 'Substrate': ((3.789636860053752+0j), 0j),
 'Substrate_bottom': ((0.595143736436588+0j), 0j)}

In [19]:
abs_dict_fluxmethod = compute_fluxes(sim)

Total Incident Power = 0.0006636046823595181
Total Reflected Power = 8.172779662924032e-05
Total Transmitted Power = 4.9367521272032344e-05
Total Absorbed Power = 0.0005325093644582455
-------------------------
Layer: ITO
Forward Top: (7.133256784779571-0.00415199595578708j)
Backward Top: (-0.1185156149453849+0.00415199595578708j)
Forward Bottom: (6.657799654778778+0.007113102097409382j)
Backward Bottom: (-0.13981974282046136-0.007113102097409382j)
Power Entering Layer: (7.273076527600033+0.0029611061416223022j)
Power Leaving Layer: (6.776315269724162+0.0029611061416223022j)
Absorbed in Layer: (4.120663709265394e-05+0j)
-------------------------
Layer: Substrate
Forward Top: (3.789636860053752+0j)
Backward Top: 0j
Forward Bottom: (0.595143736436588+0j)
Backward Bottom: 0j
Power Entering Layer: (3.789636860053752+0j)
Power Leaving Layer: (0.595143736436588+0j)
Absorbed in Layer: (0.00026498507432470413+0j)
-------------------------
Layer: Air
Forward Top: (8+0j)
Backward Top: (-0.985258

In [20]:
sim.get_field()

In [21]:
sim_proc = Simulation(simulator=sim)
freq = sim_proc.conf[('Simulation', 'params', 'frequency','value')]
try:
    Esq = sim_proc.data['normEsquared']
except KeyError:
    Esq = sim_proc.normEsquared()
abs_dict_intmethod = {}
for layer_name, layer_obj in sim_proc.layers.items():
    print("Layer: {}".format(layer_name))
    base_unit = sim_proc.conf[('Simulation', 'base_unit')]
    n_mat, k_mat = layer_obj.get_nk_matrix(freq)
    # n and k could be functions of space, so we need to multiply the
    # fields by n and k before integrating
    res = integrate(Esq*n_mat*k_mat,layer_obj, sim_proc)
    p_abs_imag = 2*np.pi*freq*consts.epsilon_0*res*base_unit
    abs_dict_intmethod[layer_name] = p_abs_imag
    disp.display_latex("$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV"+" = {}$".format(p_abs_imag), raw=True)

Layer: Air


Layer: ITO


Layer: NW_AlShell


Layer: Substrate


In [22]:
print(abs_dict_fluxmethod)
print(abs_dict_intmethod)

{'Air': 0j, 'NW_AlShell': (0.00022631765304085408+0j), 'ITO': (4.120663709265394e-05+0j), 'Substrate': (0.00026498507432470413+0j)}
{'Air': 0.0, 'NW_AlShell': 0.00030266718657491373, 'ITO': 4.0644281193719912e-05, 'Substrate': 0.00025817459345175699}


In [23]:
for key in abs_dict_fluxmethod.keys():
    fm = abs_dict_fluxmethod[key]
    im = abs_dict_intmethod[key]
    diff = fm - im
    try:
        pdiff = 100*abs(diff)/fm
    except ZeroDivisionError:
        pdiff = None
        pass
    print('-'*25)
    print("Layer: {}".format(key))
    print("Flux Method: {}".format(fm))
    print("Integral Method: {}".format(im))
    print("Diff: {}".format(diff))
    print("Percent Diff: {}".format(pdiff))

-------------------------
Layer: Air
Flux Method: 0j
Integral Method: 0.0
Diff: 0j
Percent Diff: None
-------------------------
Layer: NW_AlShell
Flux Method: (0.00022631765304085408+0j)
Integral Method: 0.00030266718657491373
Diff: (-7.634953353405965e-05+0j)
Percent Diff: (33.735562607781766+0j)
-------------------------
Layer: ITO
Flux Method: (4.120663709265394e-05+0j)
Integral Method: 4.064428119371991e-05
Diff: (5.623558989340273e-07+0j)
Percent Diff: (1.3647216531394175+0j)
-------------------------
Layer: Substrate
Flux Method: (0.00026498507432470413+0j)
Integral Method: 0.000258174593451757
Diff: (6.810480872947145e-06+0j)
Percent Diff: (2.5701375408796805+0j)


In [24]:
print(sim.data['fluxes'])

KeyError: 'fluxes'

In [25]:
print(sim.flux_dict)

{'ITO_bottom': ((6.657799654778778+0.007113102097409382j), (-0.13981974282046136-0.007113102097409382j)), 'ITO': ((7.133256784779571-0.00415199595578708j), (-0.1185156149453849+0.00415199595578708j)), 'Substrate': ((3.789636860053752+0j), 0j), 'NW_AlShell_bottom': ((4.15627843758476+0.01853459036351636j), (-0.36664157753097343-0.01853459036351636j)), 'Air': ((8+0j), (-0.9852588301655535+0j)), 'NW_AlShell': ((6.744153574990113+0.010119357499265125j), (-0.2261736630319038-0.010119357499265125j)), 'Substrate_bottom': ((0.595143736436588+0j), 0j), 'Air_bottom': ((8+0j), (-0.9852588301655535+0j))}


In [26]:
simulation = Simulation(simulator=sim)
print(simulation.data['fluxes'])
print(simulation.layers)

DEBUG:nanowire.optics.postprocess:Logger initialized


{'ITO_bottom': ((6.657799654778778+0.007113102097409382j), (-0.13981974282046136-0.007113102097409382j)), 'ITO': ((7.133256784779571-0.00415199595578708j), (-0.1185156149453849+0.00415199595578708j)), 'Substrate': ((3.789636860053752+0j), 0j), 'NW_AlShell_bottom': ((4.15627843758476+0.01853459036351636j), (-0.36664157753097343-0.01853459036351636j)), 'Air': ((8+0j), (-0.9852588301655535+0j)), 'NW_AlShell': ((6.744153574990113+0.010119357499265125j), (-0.2261736630319038-0.010119357499265125j)), 'Substrate_bottom': ((0.595143736436588+0j), 0j), 'Air_bottom': ((8+0j), (-0.9852588301655535+0j))}
OrderedDict([('Air', <nanowire.optics.utils.geometry.Layer object at 0x7f1b66a88be0>), ('ITO', <nanowire.optics.utils.geometry.Layer object at 0x7f1b66a88cc0>), ('NW_AlShell', <nanowire.optics.utils.geometry.Layer object at 0x7f1b66a886a0>), ('Substrate', <nanowire.optics.utils.geometry.Layer object at 0x7f1b66a88a20>)])


In [29]:
alist = [1., 2., 3.]
print('{}, {}, {}, {}'.format('test', *alist))

test, 1.0, 2.0, 3.0
